In [14]:
from PIL import Image, ImageOps
import numpy as np
import os

def standardize_image_smart(image_path, output_path, target_size=(64, 64)):
    # Open the image
    img = Image.open(image_path).convert('L')  # Convert to grayscale ('L')
    
    # Convert the image to a numpy array
    img_np = np.array(img)
    
    # Threshold the image to get the bounding box of the number
    threshold = 200  # Define a threshold to detect the number
    binary_img = img_np < threshold  # Pixels darker than 200 are part of the digit
    
    # Find the bounding box of the digit
    coords = np.column_stack(np.where(binary_img))
    if coords.size == 0:
        print(f"No digit found in {image_path}")
        return
    top_left = coords.min(axis=0)
    bottom_right = coords.max(axis=0)
    
    # Crop the image to the bounding box
    cropped_img_np = img_np[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    
    # Convert back to a Pillow Image
    cropped_img = Image.fromarray(cropped_img_np)
    
    # Create a new square image with the target size and paste the cropped digit into the center
    new_img = Image.new('L', target_size, color=255)  # Create a white background
    aspect_ratio = min(target_size[0] / cropped_img.width, target_size[1] / cropped_img.height)
    
    # Resize the cropped image maintaining the aspect ratio
    new_size = (int(cropped_img.width * aspect_ratio), int(cropped_img.height * aspect_ratio))
    resized_img = cropped_img.resize(new_size)
    
    # Calculate the position to center the resized image in the target size
    x_offset = (target_size[0] - resized_img.width) // 2
    y_offset = (target_size[1] - resized_img.height) // 2
    
    # Paste the resized image in the center of the new white background image
    new_img.paste(resized_img, (x_offset, y_offset))
    
    # Save the standardized image
    new_img.save(output_path)

# Apply the function to all prototype images in the local folder
prototype_folder = 'prototypes/'  # Set your folder path
output_folder = 'prototypes/standardized_prototypes/'  # Set your output folder

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for image_file in os.listdir(prototype_folder):
    if image_file.endswith('.png'):
        image_path = os.path.join(prototype_folder, image_file)
        output_path = os.path.join(output_folder, image_file)
        standardize_image_smart(image_path, output_path)
        print(f"Standardized: {image_file}")


Standardized: eight.png
Standardized: five.png
Standardized: four.png
Standardized: nine.png
Standardized: one.png
Standardized: seven.png
Standardized: six.png
Standardized: three.png
Standardized: two.png
Standardized: zero.png
